In [5]:
# Import necessary libraries
import openai
import gsearch
import json
import os
import re
import tiktoken
import time
import inspect
import callgpt
import react_chain
import compliance_checker
import excel_analysis
import repo_reader
from openpyxl import load_workbook
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter import filedialog
from tkinter import *

def count_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(text))
    return num_tokens
    
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()
os.environ["OPENAI_API_KEY"] = open_file('Keys/openai_api_key.txt')
openai.api_key = open_file('Keys/openai_api_key.txt')
openai_api_key = openai.api_key
callgpt = callgpt.Ask()
class BusinessAnalyst:
    def __init__(self):
        self.role_system = {"role": "system", "content": "You are a highly organised and detail oriented Goldman Sachs business analyst."}
        self.messages = [self.role_system]
        # INSERT INFO ABOUT SAVING THE PROMPT + COMPLETION PAIRS - FAISS? CHROMA?

    def chat_with_gpt3(self, prompt, max_retries=3, delay=2):
        user_message = {"role": "user", "content": prompt}
        new_message_tokens = count_tokens(prompt)
        total_tokens = sum(count_tokens(m["content"])
                        for m in self.messages) + new_message_tokens

        while total_tokens + 1 > 3500:
            # Remove the oldest user-assistant message pair to stay within the token limit
            self.messages.pop(1)
            self.messages.pop(1)
            total_tokens = sum(count_tokens(m["content"])
                            for m in self.messages) + new_message_tokens

        self.messages.append(user_message)
        response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages
                )

        # Retry logic
        retries = 0
        while retries < max_retries:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages,
                )
                assistant_message = {
                    "role": "assistant", "content": response.choices[0].message["content"]}
                self.messages.append(assistant_message)
                return assistant_message["content"]
            except openai.error.OpenAIError as e:
                print(f"Error occurred during API call: {e}")
                retries += 1
                if retries < max_retries:
                    time.sleep(delay)  # Delay between retries
                else:
                    raise  # Raise the exception if all retries have failed


    def process_input(self, user_query):
        prompt = f"You are an AI who wants to answer the following user query: '{user_query}' with a numbered list of tasks. Response: "
        processed_query = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Processed query:*****\n" + "\033[0m\033[0m"+processed_query)
        return processed_query

    # Not strictly speaking necessary, unless we want to do nested tasks
    def decompose_tasks(self, processed_query): 
        prompt = f"You are an AI whose job is to split the following processed query into sub-tasks and create a new numbered task list: '{processed_query}'. Response: "
        sub_tasks = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Subtasks*****\n"+"\033[0m\033[0m"+ (sub_tasks))
        return sub_tasks

    def select_task(self, sub_tasks):
        prompt = f"Select the first task to analyse from '{sub_tasks}'. Response: "
        selected_task = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Selected Tasks:*****\n"+"\033[0m\033[0m"+ (selected_task))
        return selected_task

    def choose_best_tool(self, task):
        prompt = f"""You are an AI that has to select the best tool to use to solve the following task: '{task}'.
        Please respond in the format 'Action: [tool_name] only including items from this list - Action: (gsearch|repo_reader|ask_user_input|agent_discussion|agent_report|analyse_excel.
        This is how you make your choice:
        If you think talking it through might help: agent_discussion
        If you want to write a report: agent_report
        If you need to seach the web: gsearch
        If you need user input: ask_user_input
        The response should contain absolutely no other words. 
        The default option should be 'Action: gsearch' """
        response = self.chat_with_gpt3(prompt)
        tool_name = response.split(': ')[1]  # Split the response and retrieve the tool name

        print(f"GPT-3 Response: {response}")  # Print the response from GPT-3
        
        tool_re = re.compile(r"'\s*(.*?)\s*'")
        tool_match = tool_re.search(response)
        if not tool_match:
            # Try to match the response without single quotes
            tool_re = re.compile(r"Action:\s*(gsearch|repo_reader|ask_user_input|agent_discussion|agent_report|analyse_excel)", re.IGNORECASE)
            tool_match = tool_re.search(response)

        if tool_match:
            best_tool = tool_match.group(1)
            print(f"Extracted tool from GPT-3 response: {best_tool}")  # Print the extracted tool
            return best_tool.strip().lower().replace(" ", "_")
        else:
            print(f"Unexpected GPT-3 response: {response}")  # Add this line to print the unexpected response
            raise ValueError("GPT-3 did not provide a valid tool name in its response.")
        
    def ask_user_input(self, prompt):
        user_input = input("What should I do\n" + prompt)
        return user_input

    def agent_discussion(self, prompt):
        discussion = agents_discussion.generate_output(prompt)
        return discussion

    def agent_report(self, prompt):
        report = agents_report.create_joint_letter(prompt)
        return report
    
    def execute_tasks(self, selected_task, max_retries=3):
        results = []
        # ADD MORE FUNCTIONS
        function_mapping = {
            "gsearch": gsearch.execute,
            "repo_reader": repo_reader.main,
            "analyse_excel": self.analyse_excel,
            "ask_user_input": self.ask_user_input,
            "agent_discussion": self.agent_discussion,
            "agent_report": self.agent_report
        }

        retries = 0
        while retries < max_retries:
            best_tool = self.choose_best_tool(selected_task)
            if best_tool in function_mapping:
                tool_function = function_mapping[best_tool]
                break
            else:
                print(f"Invalid tool selected: {best_tool}. Retrying...")
                retries += 1

        if retries == max_retries:
            raise ValueError("Failed to select a valid tool after multiple retries.")

        prompt = f"{best_tool}: {selected_task}"
        result = tool_function(prompt)  # Use the selected function to solve the task
        print("\033[92m\033[1m" + "\n*****Results Of Execution*****\n" + "\033[0m\033[0m" + str(result))
        return result
        # result = gsearch.execute(prompt)  # Use the selected function to solve the task
        # print("\033[92m\033[1m" + "\n*****Results Of Execution*****\n" + "\033[0m\033[0m" + str(result))
        # result = self.chat_with_gpt3(prompt)
        return result

    def analyze_results(self, results):
        prompt = f"You are an AI focused on analysing the results from executing the task: '{results}'. Please critically explain and appraise the output. Answer what are avenues to further explore and understand from this. "
        analysis = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Analysis*****\n"+"\033[0m\033[0m" + (analysis))
        return analysis

    def refine_tasks(self, analysis):
        prompt = f"Considering the results, let's refine and create a new list of tasks based on the following analysis: {analysis}. The response should be in a numbered list with one sentence each. Response: "
        new_tasks = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Refined new Tasks*****\n"+"\033[0m\033[0m"+ (new_tasks))
        return new_tasks

    def generate_output(self, analysis):
        prompt = f"You are an analyst who will create a readable document from the whole discussion summarising it and clearly articulating key theories, ideas and actions to be done. Please generate a user-friendly output based on the following analysis: {analysis}. Response: "
        output = self.chat_with_gpt3(prompt)
        print("\033[92m\033[1m"+"\n*****Generated Output*****\n"+"\033[0m\033[0m"+ (output))
        return output

    def run_analysis(self, user_query):
        processed_query = self.process_input(user_query)
        sub_tasks = self.decompose_tasks(processed_query)
        selected_tasks = self.select_task(sub_tasks)
        results = self.execute_tasks(selected_tasks)
        analysis = self.analyze_results(results)
        new_tasks = self.refine_tasks(analysis)
        ques = input("Do you want to continue - Y or N? ")
        while ques == "Y":
            sub_tasks = self.process_input(new_tasks)
            selected_tasks = self.select_task(sub_tasks)
            results = self.execute_tasks(selected_tasks)
            analysis = self.analyze_results(results)
            new_tasks = self.refine_tasks(analysis)
            ques = input("Do you want to continue - Y or N? ")
        if ques == "N":
            print("\033[92m\033[1m" + "\n*****Alright! Final Results Coming Up*****\n" + "\033[0m\033[0m")
        else:
            print("\n\n*******Dude! I said Y or N!*******\n\n")
        output = self.generate_output(analysis)
        return output
    
    def get_excel_file_path(self):
        root = Tk()
        root.withdraw()  # Hide the Tkinter root window
        file_path = filedialog.askopenfilename(initialdir="/", title="Select a File", filetypes=[("Excel Files", "*.xlsx")])
        print(file_path)
        root.destroy()  # Close the Tkinter root window
        return file_path

    def analyse_excel(self, prompt):
        print("\033[92m\033[1m" + "\n*****Excel Analysis*****\n" + "\033[0m\033[0m")
        functions_list = [func for func in inspect.getmembers(excel_analysis, inspect.isfunction)]

        filepath = self.get_excel_file_path()
        results = excel_analysis.main(filepath)

        for recursion_count in range(5):
            analysis = self.analyze_results(results)
            # Process the best result and create tasks
            prompt = f"As a top inventory analyst at Shopify or a similar institution, when presented with an Excel sheet functioning as an inventory management system, what tests and hypotheses would you employ to determine if there is adequate stock remaining for specific SKUs? {analysis}. Write your response as a numbered task list: "
            output = self.chat_with_gpt3(prompt)

            # Evaluate the output
            evaluation_prompt = f"How good is the following analysis: '{output}'. Please rate it on a scale of 1 to 10, with 10 being the best: "
            results = self.decompose_tasks(output)

        sub_tasks = self.process_input(results)

        final_results = []
        for task in sub_tasks:
            result = self.execute_tasks(task)
            final_results.append(result)

        # Combine the results and analyze them
        final_result = ' '.join(final_results)
        analysis = self.analyze_results(final_result)

        return final_result


ba = BusinessAnalyst()
result = None

while True:
    try:
        user_query = input("""Choose one. What would you like to do? \n
                    1. Ask something you're curious about, run tasks \n
                    2. Figure out something about your docs \n
                    3. Figure out what's inside a python repo\n
                    4. Simulate a dialogue between two people \n
                    5. Let's write a report together! \n
                    6. Analyse an excel! \n
                    7. Write a short script. \n
                    00. Exit the prompt \n
                    """)
        if user_query == "1":
            prompt = input("Enter your query: ")
            result = ba.run_analysis(prompt)
        elif user_query == "2":
            # Call the function to figure out something about your docs
            # Replace this line with the appropriate function call
            result = "\n\nFigure out something about your docs (function not implemented)\n\n"
        elif user_query == "3":
            # Call the function to figure out what's inside a python repo
            # Replace this line with the appropriate function call
            result = "\n\nFigure out what's inside a python repo (function not implemented)\n\n"
        elif user_query == "4":
            # Call the function to simulate a dialogue between two people
            import agents_discussion
            result = agents_discussion.generate_output(user_query)
            print("\n\nHere's a dialogue!\n\n")
        elif user_query == "5":
            # Call the function to write a report together
            import agents_report
            result = agents_report.create_joint_letter(user_query)
            print("\n\nHere's the report!\n\n")
        elif user_query == "6":
            # Call the function to analyze an excel
            result = ba.analyse_excel("Analyze an excel sheet")
        elif user_query == "7":
            # Call the function to write a short script
            # Replace this line with the appropriate function call
            result = "\n\nWrite a short script (function not implemented)\n\n"
        elif user_query == "00":
            result = "\n\nSorry to see you go !!\n\n"
        else:
            print("\n\nInvalid option. Please choose a number between 1 and 7.\n\n")
    except TypeError as e:
        print(f"An error occurred: {e}\nPlease try again.")
    except ValueError as e:
        print(f"An error occurred: {e}\nPlease try again.")

if result:
    print(result)



*****Processed query:*****
Sure, here is a list of tasks that a VC analyst might perform when evaluating whether to invest in an AI software startup:

1. Analyze the market potential for the startup's AI software by researching the demand for similar products and assessing the competitive landscape.
2. Evaluate the startup's technology and intellectual property, including patent filings and proprietary software.
3. Assess the startup's team, including the founders' experience, skills, and ability to execute on the company's business plan.
4. Review the startup's financial projections, including revenue and cost projections, burn rate, and runway.
5. Conduct due diligence on the company's financials, including revenue sources, expenses, and potential liabilities.
6. Evaluate the startup's fundraising history, including previous rounds of financing, institutional investors, and any previous exits.
7. Determine the startup's key performance indicators (KPIs) and metrics, and evaluate how

KeyboardInterrupt: 